In [ ]:
# Implement sequence modeling of stock prices using LSTM layer in Bitcoin minute price data
# Comparing accuracy of different timesteps of 60, 120 and 180 

In [ ]:
import tensorflow as tf

tf.test.gpu_device_name()

'/device:GPU:0'

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [3]:
#changing the working directory

%cd /content/gdrive/My Drive/Kaggle


/content/gdrive/My Drive/Kaggle


In [4]:
% pwd

#Check the present working directory using pwd command

'/content/gdrive/My Drive/Kaggle'

In [5]:
!ls

 0.2.0.zip				  Recommendation_df_test1.csv
 Apple-Twitter-Sentiment-DFE.csv	  Recommendation_df_test.csv
 apple-twitter-sentiment-texts.csv	  Recommendation_df_train1.csv
 Bitcoin_prediction_One_hot_encoded.csv   Recommendation_df_train.csv
 BitCoinPrice.csv			  sampleSubmission.csv
 Bitcoin_tweet_sentiment.csv		  sampleSubmission.csv.zip
 BTCUSD_1hr.csv				  sampleSubmission_v2.csv.zip
 BTCUSD_1min.csv			  saved_models
 BTCUSD_day.csv				  Song_Recommendation_Model.h5
 Credit_Card_Autoencoder_Model.h5	  submission.csv
 creditcard.csv				  test.csv
 decoder_layer.png			  test_embed_df.csv
 decoder.png				  test.tsv
 encoder_layer.png			  train.csv
 encoder.png				  train_embed_df.csv
 FastText				  train-sample.csv.zip
'Harry Potter 1.csv'			  train.tsv
'Harry Potter 2.csv'			  train_v2.csv
'Harry Potter 3.csv'			  transformer.png
 kaggle.json				  tweetsfinal1.csv
 Movie_Conversation.csv			  tweetsfinal2.csv
 movie_conversations.txt		  tweetsfinal3.csv
 Movie_Dialogue_Model1.h5		  tw

In [41]:
import pandas as pd 

df_bc1 = pd.read_csv('BTCUSD_1min.csv')
df_bc1

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1586390400000,2020-04-09 00:00:00,BTCUSD,7369.60,7369.60,7348.47,7350.41,7.733528
1,1586390340000,2020-04-08 23:59:00,BTCUSD,7364.37,7369.60,7364.37,7369.60,1.310954
2,1586390280000,2020-04-08 23:58:00,BTCUSD,7360.89,7367.18,7360.89,7364.37,1.018774
3,1586390220000,2020-04-08 23:57:00,BTCUSD,7357.62,7366.12,7357.62,7360.89,0.026251
4,1586390160000,2020-04-08 23:56:00,BTCUSD,7370.71,7370.71,7357.62,7357.62,0.444867
...,...,...,...,...,...,...,...,...
2283514,1444311840,2015-10-08 13:44:00,BTCUSD,242.96,242.96,242.96,242.96,0.033491
2283515,1444311780,2015-10-08 13:43:00,BTCUSD,242.95,242.96,242.95,242.96,0.010000
2283516,1444311720,2015-10-08 13:42:00,BTCUSD,242.95,242.95,242.95,242.95,0.000000
2283517,1444311660,2015-10-08 13:41:00,BTCUSD,242.50,242.95,242.50,242.95,0.001000


In [42]:
# Reversing the dataframe to make the prices in increasing order of timesteps

df_bc1 = df_bc1.iloc[::-1]
df_bc1

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
2283518,1444311600,2015-10-08 13:40:00,BTCUSD,0.00,242.50,0.00,242.50,0.050000
2283517,1444311660,2015-10-08 13:41:00,BTCUSD,242.50,242.95,242.50,242.95,0.001000
2283516,1444311720,2015-10-08 13:42:00,BTCUSD,242.95,242.95,242.95,242.95,0.000000
2283515,1444311780,2015-10-08 13:43:00,BTCUSD,242.95,242.96,242.95,242.96,0.010000
2283514,1444311840,2015-10-08 13:44:00,BTCUSD,242.96,242.96,242.96,242.96,0.033491
...,...,...,...,...,...,...,...,...
4,1586390160000,2020-04-08 23:56:00,BTCUSD,7370.71,7370.71,7357.62,7357.62,0.444867
3,1586390220000,2020-04-08 23:57:00,BTCUSD,7357.62,7366.12,7357.62,7360.89,0.026251
2,1586390280000,2020-04-08 23:58:00,BTCUSD,7360.89,7367.18,7360.89,7364.37,1.018774
1,1586390340000,2020-04-08 23:59:00,BTCUSD,7364.37,7369.60,7364.37,7369.60,1.310954


In [43]:
# Finding out percentage increase or decrease 

df_bc1['Pct_change'] = df_bc1['Close'].pct_change()
df_bc1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,Pct_change
2283518,1444311600,2015-10-08 13:40:00,BTCUSD,0.00,242.50,0.00,242.50,0.050000,NaN
2283517,1444311660,2015-10-08 13:41:00,BTCUSD,242.50,242.95,242.50,242.95,0.001000,0.001856
2283516,1444311720,2015-10-08 13:42:00,BTCUSD,242.95,242.95,242.95,242.95,0.000000,0.000000
2283515,1444311780,2015-10-08 13:43:00,BTCUSD,242.95,242.96,242.95,242.96,0.010000,0.000041
2283514,1444311840,2015-10-08 13:44:00,BTCUSD,242.96,242.96,242.96,242.96,0.033491,0.000000
...,...,...,...,...,...,...,...,...,...
4,1586390160000,2020-04-08 23:56:00,BTCUSD,7370.71,7370.71,7357.62,7357.62,0.444867,-0.001776
3,1586390220000,2020-04-08 23:57:00,BTCUSD,7357.62,7366.12,7357.62,7360.89,0.026251,0.000444
2,1586390280000,2020-04-08 23:58:00,BTCUSD,7360.89,7367.18,7360.89,7364.37,1.018774,0.000473
1,1586390340000,2020-04-08 23:59:00,BTCUSD,7364.37,7369.60,7364.37,7369.60,1.310954,0.000710


In [44]:
# Putting 1 if percentage increase and 0 if percentage decrease/stays same
# Idea being invest in stock if 1 and dont invest / short the stock if 0

df_bc1['Increase'] = df_bc1['Pct_change'].apply(lambda x: 1 if x > 0 else 0)
df_bc1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,Pct_change,Increase
2283518,1444311600,2015-10-08 13:40:00,BTCUSD,0.00,242.50,0.00,242.50,0.050000,NaN,0
2283517,1444311660,2015-10-08 13:41:00,BTCUSD,242.50,242.95,242.50,242.95,0.001000,0.001856,1
2283516,1444311720,2015-10-08 13:42:00,BTCUSD,242.95,242.95,242.95,242.95,0.000000,0.000000,0
2283515,1444311780,2015-10-08 13:43:00,BTCUSD,242.95,242.96,242.95,242.96,0.010000,0.000041,1
2283514,1444311840,2015-10-08 13:44:00,BTCUSD,242.96,242.96,242.96,242.96,0.033491,0.000000,0
...,...,...,...,...,...,...,...,...,...,...
4,1586390160000,2020-04-08 23:56:00,BTCUSD,7370.71,7370.71,7357.62,7357.62,0.444867,-0.001776,0
3,1586390220000,2020-04-08 23:57:00,BTCUSD,7357.62,7366.12,7357.62,7360.89,0.026251,0.000444,1
2,1586390280000,2020-04-08 23:58:00,BTCUSD,7360.89,7367.18,7360.89,7364.37,1.018774,0.000473,1
1,1586390340000,2020-04-08 23:59:00,BTCUSD,7364.37,7369.60,7364.37,7369.60,1.310954,0.000710,1


In [45]:
# Timesteps or window size = 60

import numpy as np

total = df_bc1['Increase'].tolist()
total = np.array(total)
total1 = total[:2283480]
len(total1)


2283480

In [46]:
total1 = total1.reshape(-1,60,1)
total1.shape

(38058, 60, 1)

In [47]:
split = int(0.8*len(total1))
split

30446

In [48]:
xx = total1[:split]
xx_test = total1[split:]

In [49]:
xx.shape

(30446, 60, 1)

In [50]:
xx_test.shape

(7612, 60, 1)

In [51]:
[x, y] = np.hsplit(xx, np.array([59]))

In [52]:
x.shape

(30446, 59, 1)

In [53]:
y.shape

(30446, 1, 1)

In [54]:
[x_test, y_test] = np.hsplit(xx_test, np.array([59]))

In [32]:
xpoints = np.array(range(len(y_test)))
xpoints

array([   0,    1,    2, ..., 7609, 7610, 7611])

In [1]:
# import matplotlib.pyplot as plt
# import numpy as np

# ypoints = y_test.reshape(-1)
# # xpoints = range(len(y_test))
# # plt.plot(ypoints, linestyle = 'dotted')
# # plt.show()

# plt.xlim(6000, 6100)
# plt.scatter(xpoints, ypoints)

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam

n_steps = 59
n_features = 1

model1 = Sequential()
model1.add(LSTM(50, return_sequences=True, input_shape=(59,1), activation='relu'))
model1.add(LSTM(50, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 59, 50)            10400     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


In [56]:
model1.fit(x, y, batch_size = 32, epochs=5, verbose = 1)

Epoch 1/5
952/952 [==============================] - 58s 59ms/step - loss: 0.4478 - accuracy: 0.7905
Epoch 2/5
952/952 [==============================] - 55s 58ms/step - loss: 0.4373 - accuracy: 0.7910
Epoch 3/5
952/952 [==============================] - 56s 58ms/step - loss: 0.4349 - accuracy: 0.7912
Epoch 4/5
952/952 [==============================] - 55s 58ms/step - loss: 0.4345 - accuracy: 0.7910
Epoch 5/5
952/952 [==============================] - 55s 58ms/step - loss: 0.4343 - accuracy: 0.7907


In [57]:
model1.evaluate(x_test, y_test, verbose = 1)

238/238 [==============================] - 4s 16ms/step - loss: 0.6766 - accuracy: 0.6123


[0.6765562295913696, 0.6123226284980774]

In [58]:
model1.metrics_names

['loss', 'accuracy']

In [59]:
# Timesteps or window size = 120

total = df_bc1['Increase'].tolist()
total = np.array(total)
total1 = total[:2283480]
len(total1)


2283480

In [60]:
total1 = total1.reshape(-1,120,1)
total1.shape

(19029, 120, 1)

In [61]:
split = int(0.8*len(total1))
split

15223

In [62]:
xx = total1[:split]
xx_test = total1[split:]

In [63]:
xx.shape

(15223, 120, 1)

In [64]:
xx_test.shape

(3806, 120, 1)

In [65]:
[x, y] = np.hsplit(xx, np.array([119]))

In [66]:
x.shape

(15223, 119, 1)

In [67]:
y.shape

(15223, 1, 1)

In [68]:
[x_test, y_test] = np.hsplit(xx_test, np.array([119]))

In [69]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam

n_steps = 119
n_features = 1

model1 = Sequential()
model1.add(LSTM(50, return_sequences=True, input_shape=(119,1), activation='relu'))
model1.add(LSTM(50, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 119, 50)           10400     
                                                                 
 lstm_3 (LSTM)               (None, 50)                20200     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


In [71]:
model1.fit(x, y, batch_size = 32, epochs=15, verbose = 1)

Epoch 1/15
476/476 [==============================] - 53s 112ms/step - loss: 0.4430 - accuracy: 0.7885
Epoch 2/15
476/476 [==============================] - 53s 112ms/step - loss: 0.4417 - accuracy: 0.7886
Epoch 3/15
476/476 [==============================] - 54s 114ms/step - loss: 0.4422 - accuracy: 0.7889
Epoch 4/15
476/476 [==============================] - 53s 112ms/step - loss: 0.4412 - accuracy: 0.7891
Epoch 5/15
476/476 [==============================] - 54s 113ms/step - loss: 0.4401 - accuracy: 0.7877
Epoch 6/15
476/476 [==============================] - 54s 113ms/step - loss: 0.4832 - accuracy: 0.7892
Epoch 7/15
476/476 [==============================] - 54s 113ms/step - loss: 0.4401 - accuracy: 0.7887
Epoch 8/15
476/476 [==============================] - 53s 112ms/step - loss: 0.4409 - accuracy: 0.7888
Epoch 9/15
476/476 [==============================] - 54s 113ms/step - loss: 0.4392 - accuracy: 0.7888
Epoch 10/15
476/476 [==============================] - 54s 113ms/step - l

In [72]:
model1.evaluate(x_test, y_test, verbose = 1)

119/119 [==============================] - 4s 30ms/step - loss: nan - accuracy: 0.6148


[nan, 0.6148186922073364]

In [73]:
model1.metrics_names

['loss', 'accuracy']

In [74]:
# Timesteps or window size = 180

total = df_bc1['Increase'].tolist()
total = np.array(total)
total1 = total[:2283480]
len(total1)


2283480

In [75]:
total1 = total1.reshape(-1,180,1)
total1.shape

(12686, 180, 1)

In [76]:
split = int(0.8*len(total1))
split

10148

In [77]:
xx = total1[:split]
xx_test = total1[split:]

In [78]:
xx.shape

(10148, 180, 1)

In [79]:
xx_test.shape

(2538, 180, 1)

In [80]:
[x, y] = np.hsplit(xx, np.array([179]))

In [81]:
x.shape

(10148, 179, 1)

In [82]:
y.shape

(10148, 1, 1)

In [83]:
[x_test, y_test] = np.hsplit(xx_test, np.array([179]))

In [84]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam

n_steps = 179
n_features = 1

model1 = Sequential()
model1.add(LSTM(50, return_sequences=True, input_shape=(179,1), activation='relu'))
model1.add(LSTM(50, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 179, 50)           10400     
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


In [85]:
model1.fit(x, y, batch_size = 32, epochs=15, verbose = 1)

Epoch 1/15
318/318 [==============================] - 56s 169ms/step - loss: 5990727.5000 - accuracy: 0.7854
Epoch 2/15
318/318 [==============================] - 54s 169ms/step - loss: 759816716288.0000 - accuracy: 0.7857
Epoch 3/15
318/318 [==============================] - 54s 168ms/step - loss: 0.5364 - accuracy: 0.7858
Epoch 4/15
318/318 [==============================] - 53s 166ms/step - loss: 0.5199 - accuracy: 0.7855
Epoch 5/15
318/318 [==============================] - 53s 166ms/step - loss: 0.4990 - accuracy: 0.7858
Epoch 6/15
318/318 [==============================] - 53s 165ms/step - loss: 0.4941 - accuracy: 0.7858
Epoch 7/15
318/318 [==============================] - 53s 166ms/step - loss: 0.4913 - accuracy: 0.7858
Epoch 8/15
318/318 [==============================] - 52s 165ms/step - loss: 0.4890 - accuracy: 0.7858
Epoch 9/15
318/318 [==============================] - 51s 162ms/step - loss: 0.4869 - accuracy: 0.7858
Epoch 10/15
318/318 [==============================] - 5

In [86]:
model1.evaluate(x_test, y_test, verbose = 1)

80/80 [==============================] - 3s 39ms/step - loss: 0.6949 - accuracy: 0.6190


[0.6948772072792053, 0.6189913153648376]

In [87]:
model1.metrics_names

['loss', 'accuracy']